In [1]:
from datetime import datetime,timedelta #import the datetime module
import pandas as pd
import random
import time
import numpy as np
import os
import json
import sys
import asyncio
import aiohttp
import winsound



In [26]:
#Set Pandas display options
pd.options.display.float_format = '{:,.2f}'.format
pd.set_option('display.max_columns', None)

#set the subdirectory to save the tables
savedir=os.getcwd()+'\\datatables\\gamelogs\\'

In [27]:
"""Used to make a beep noise to indicate the script is complete. This will only work in Windows"""
def beep_sound():
    duration = 2000  # milliseconds
    freq = 1500  # Hz
    winsound.Beep(freq, duration)

In [28]:
'''A dictionary that is used to map the visitor team ID to the abbreviation'''
team_dict={1610612737: 'ATL',
 1610612738: 'BOS',
 1610612751: 'BKN',
 1610612766: 'CHA',
 1610612741: 'CHI',
 1610612739: 'CLE',
 1610612742: 'DAL',
 1610612743: 'DEN',
 1610612765: 'DET',
 1610612744: 'GSW',
 1610612745: 'HOU',
 1610612754: 'IND',
 1610612746: 'LAC',
 1610612747: 'LAL',
 1610612763: 'MEM',
 1610612748: 'MIA',
 1610612749: 'MIL',
 1610612750: 'MIN',
 1610612740: 'NOP',
 1610612752: 'NYK',
 1610612760: 'OKC',
 1610612753: 'ORL',
 1610612755: 'PHI',
 1610612756: 'PHX',
 1610612757: 'POR',
 1610612758: 'SAC',
 1610612759: 'SAS',
 1610612761: 'TOR',
 1610612762: 'UTA',
 1610612764: 'WAS'}

In [33]:
#if the season is in progress filter out any games that have not taken place from the pull
def current_year_filter(year_to_pull,game_list):
    df_data=pd.read_csv(f'{savedir}{year_to_pull}_schedule.csv',names=['game_id', 'game_date'],dtype={'game_id': object})
    df_data['game_date'] =  pd.to_datetime(df_data['game_date'], format='%Y-%m-%d')
    df_data=df_data[df_data.game_date < (datetime.today() +timedelta(days=-1))]
    return df_data.game_id

In [55]:
#This function sets the upper bound on how many queries to send per year depending on the season
def total_games(season):
    if season == 1998:
        total_g=725
    elif season ==2011:
        total_g=1025
    elif season <=1987:
        total_g=943
    elif season <=1994:
        total_g=1107
    elif season <=1994:
        total_g=1189
    else:
        total_g=1230
    return total_g

In [13]:
def create_basic_box_df(basic_box):

    # the data for the basic box score data results (basic_box[0]['resultSets'][0]['rowSet']) --> first list is for the games
    '''basic_box[2]['parameters']['GameID'] get the column for the gameID '''
    '''basic_box[0]['resultSets'][0]['rowSet'] the results for a single game '''

    df_data=[]
    errors=[] #log rows with errors
    count=0
    
    for i in basic_box:
        try: #this only works if there are results
            for j in range(len(i['resultSets'][0]['rowSet'])):
                df_data.append(i['resultSets'][0]['rowSet'][j]) #note last index is the player
                count+=1
        except TypeError:
            errors.append(count)
            count+=1
    print(f'there were: {len(errors)} errors in the basic box score data')
    df_basic_box = pd.DataFrame(df_data,columns=basic_box[0]['resultSets'][0]['headers'])
    return(df_basic_box)


In [14]:
def create_adv_box_df(adv_box):

    # the data for the adv box score data results (adv_box[0]['resultSets'][0]['rowSet']) --> first list is for the games

    df_data=[]
    errors=[] #log rows with errors
    count=0
    
    for i in adv_box:
        try:
            for j in range(len(i['resultSets'][0]['rowSet'])):
                df_data.append(i['resultSets'][0]['rowSet'][j]) #note last index is the player
                count+=1
        except TypeError:
            errors.append(count)
            count+=1
    print(f'there were: {len(errors)} errors in the advanced box score data')
    df_adv_box = pd.DataFrame(df_data,columns=adv_box[0]['resultSets'][0]['headers'])
    return(df_adv_box)

In [15]:
def create_summ_box_df(summ_box):

    # the data for the adv box score data results (adv_box[0]['resultSets'][0]['rowSet']) --> first list is for the games

    df_data=[]
    errors=[] #log rows with errors
    count=0
    
    try:
        for i in range(0,len(summ_box)):
            df_data.append(summ_box[i]['resultSets'][0]['rowSet'][0])
            count+=1
    except TypeError:
        errors.append(count)
        count+=1
    print(f'there were: {len(errors)} errors in the summary box score data')
    df_summ_box = pd.DataFrame(df_data,columns=summ_box[0]['resultSets'][0]['headers'])
    return(df_summ_box)

In [16]:
async def get_json(client,url,headers,params=''):
    async with client.get(url,params=params,headers=headers) as response:
        try:
            assert response.status==200
            ret=await response.json()
            return ret
        except AssertionError:
            pass

async def response_basic(wait_base,client,headers,payload_basic,game):
    wait_t=random.uniform(0,wait_base)
    await asyncio.sleep(wait_t)
    payload_basic['GameID']=game
    url_basic='https://stats.nba.com/stats/boxscoretraditionalv2'
    response_basic= await get_json(client,url_basic,headers,payload_basic)
    return response_basic
    
    
async def response_basic_summ(wait_base,client,headers,game):
    wait_t=random.uniform(0,wait_base)
    await asyncio.sleep(wait_t)
    url_summ='https://stats.nba.com/stats/boxscoresummaryv2?GameID=' +game 
    response_summ= await get_json(client,url_summ,headers)
    return response_summ

async def response_adv(wait_base,client,headers,payload_basic,game):
    wait_t=random.uniform(0,wait_base)
    await asyncio.sleep(wait_t)
    payload_basic['GameID']=game
    url_adv='https://stats.nba.com/stats/boxscoreadvancedv2' 
    response_adv= await get_json(client,url_adv,headers,payload_basic)
    return response_adv


async def main(game_list_key,year_param,season_param,wait_base):
    wait_base=wait_base
    start_time = time.time()
    #the headers to pass in the query
    headers = {
    'Host': 'stats.nba.com',
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:71.0) Gecko/20100101 Firefox/71.0',
    'Accept': 'application/json, text/plain, */*',
    'Accept-Language': 'en-US,en;q=0.5',
    'Referer': 'https://stats.nba.com/',
    'Accept-Encoding': 'gzip, deflate, br',
    'Connection': 'keep-alive',
        }

    '''the query parameters for the basic and advanced stats. The game ID needs to be revised in each query. The other parameters appear to be work if they remain static
    the season param is required for queries prior to 1999-20
    '''
    payload_basic={'EndPeriod': '10',
    'EndRange': '28800',
    'GameID': '0021400340',
    'RangeType': '0',
    'Season': season_param,
    'SeasonType': 'Regular Season',
    'StartPeriod': '1',
    'StartRange': '0'}    
    payload_basic['Season']=season_param
    
    conn= aiohttp.TCPConnector(limit=100)
    
    async with aiohttp.ClientSession(connector=conn) as client: #create the client sesson object that persists across requests
        '''Each of these calls the corotines to pull the basic box score stats, advanced score stats, and summary data that will be used to create the dataframe for the season'''
        basic_box_data=[asyncio.create_task(response_basic(wait_base,client,headers,payload_basic,game)) for game in game_list_key]
        summ_box_data=[asyncio.create_task(response_basic_summ(wait_base,client,headers,game)) for game in game_list_key]
        adv_box_data=[asyncio.create_task(response_adv(wait_base,client,headers,payload_basic,game)) for game in game_list_key]
            
        '''The await...gather ensures all of the queries are complete before the function returns the list of JSONs back to the main program'''    
        basic_box = await asyncio.gather(*basic_box_data, return_exceptions=True)
        summ_box = await asyncio.gather(*summ_box_data, return_exceptions=True)
        adv_box = await asyncio.gather(*adv_box_data, return_exceptions=True)
        print("it took --- %s seconds ---" % (time.time() - start_time),'to go through:'+str(len(game_list_key))+' games')
        return(basic_box,summ_box,adv_box)

    

    





In [62]:
"""This function builds the final dataframe and saves for the entire season"""
def build_df(basic_box,adv_box,summ_box,year_param):
    #convert the JSON into Dataframes
    df_basic_box=create_basic_box_df(basic_box)
    df_adv_box=create_adv_box_df(adv_box)
    df_summ_box=create_summ_box_df(summ_box)
    check=0 #this needs to change to 1 to flag some of the additional transformations. It prevents merge errors when there is no advanced or basic box score data
    #merge the dataframes and save to hdf
    
    if df_adv_box.empty == False:
        df_adv_box.drop(columns=['E_USG_PCT','E_PACE','E_OFF_RATING','E_DEF_RATING','E_NET_RATING'],inplace=True)
        df_total_year= df_basic_box.merge(df_adv_box, left_on=['GAME_ID','PLAYER_ID'], right_on=['GAME_ID','PLAYER_ID'],suffixes=('', '_delete'),how='left') #merge basic and advanced stats using the player as the key
        check=1 #flag for transformation code
    else:
        df_total_year= df_basic_box #if the join fails because there are no advanced stats
    
    if df_summ_box.empty == False:
        df_total_year=df_total_year.merge(df_summ_box, left_on='GAME_ID', right_on='GAME_ID',suffixes=('', '_delete'),how='left') #merge basic and advanced stats with box_score summary data using the player as the key
        check=1 #flag for transformation code
    
    
    if check == 1:
        df_total_year=df_total_year.loc[:,~df_total_year.columns.str.endswith('_delete')] #remove the duplicated columns
        
    df_total_year['SEASON']=year_param #add the season
    df_total_year['HM_AW']=np.where(df_total_year.TEAM_ID==df_total_year.HOME_TEAM_ID,'H','A') #mark as home or away  
    
    df_total_year['OPP']=np.where(df_total_year.TEAM_ID==df_total_year.HOME_TEAM_ID,df_total_year.VISITOR_TEAM_ID,df_total_year.HOME_TEAM_ID) #mark as home or away
    df_total_year['OPP']=df_total_year['OPP'].map(team_dict)  #get the abb for the visiting team
    
    
    #Create a pivot table that tracks which team won or lost as a boolean value then merge/join into the original table so there is a filter on if a player won or lost a specific game
    df_win_lose=(df_total_year.groupby(['GAME_ID','HM_AW'])['PTS'].sum().reset_index()).pivot(index='GAME_ID',columns='HM_AW',values='PTS')
    df_win_lose['WINNER']=np.where(df_win_lose.H>df_win_lose.A,'H','A') #mark home or away as winner
    df_total_year= df_total_year.merge(df_win_lose, left_on='GAME_ID', right_on='GAME_ID',suffixes=('', '_delete'))
    df_total_year['PLAYER_WIN_OR_LOSE']=np.where(df_total_year.WINNER==df_total_year.HM_AW,1,0) #mark home or away as winner

    #Flag unncessary columns and delete them.
    #For some of the advanced stats there is a column with "e_" as a prefix.
    #So if there is "DEF_RATING" and "E_DEF_RATING" NBA uses the first one so we will drop those columns
    int_list= ['PLAYER_ID','FGM', 'FGA', 'FG3M','FG3A', 'FTM', 'FTA', 'OREB', 'DREB', 'REB', 'AST','STL', 'BLK', 'TO', 'PF', 'PTS', 'PLUS_MINUS'] #list of columns to convert to INT
    df_total_year[int_list]=df_total_year[int_list].apply(pd.to_numeric,errors='coerce')    
    
    drop_col=['GAME_SEQUENCE', 'GAME_STATUS_ID', 'GAME_STATUS_TEXT',
           'GAMECODE', 'LIVE_PERIOD',
           'LIVE_PC_TIME', 'NATL_TV_BROADCASTER_ABBREVIATION',
           'LIVE_PERIOD_TIME_BCAST', 'WH_STATUS','HOME_TEAM_ID','VISITOR_TEAM_ID','TEAM_CITY','A','H','FG_PCT','FG3_PCT','FT_PCT']
    
    df_total_year.drop(columns=drop_col,inplace=True)
    
    return df_total_year

    

In [ ]:
#first_year=int(input('What is the first year of the season to pull:'))
#last_year=int(input('What is the last year of the season to pull:'))
first_year=int(input('Pick the first season to scrape:'))
last_year=int(input('Pick the last season to scrape:'))
wait_base=int(input('~how many seconds should the requests take in total?'))
years_to_pull=[x for x in range(first_year,last_year+1)] #create a list with the years to loop through

print(f'The following years were selected: {years_to_pull}')

for year_to_pull in years_to_pull:
    year_param=((str(year_to_pull)+'-'+str((int(year_to_pull)+1))))
    season_param=((str(year_to_pull)+'-'+str((int(year_to_pull)+1))[2:4])) #season parameter for the query
    game_list_key =[]
    game_range=total_games(year_to_pull)
    game_list=['002'+str(year_to_pull)[2:4] + str(i).zfill(5) for i in range(1,game_range+1)] #002 is the prefix, plus last two digits of the year+games 1-1230 padded to 5 digits
    #call a function to prevent querying any games that haven't taken place during the current season
    if year_to_pull == datetime.now().year:
        game_list=current_year_filter(year_to_pull,game_list)
    
    #for game_list_key in game_list[5]:
    start_time = time.time()
    #run the function that pulls the API results
    basic_box,summ_box,adv_box= await main(game_list,year_to_pull,season_param,wait_base) #this would be asyncio.run(main(client)) in plain Python(https://bit.ly/36MtBDI)
    
    
    df_total_year=build_df(basic_box,adv_box,summ_box,year_param)
    df_total_year.to_parquet(f'{savedir}df_{year_param}_data.gzip',compression='gzip')
    beep_sound() #make the computer beep when the script is finished running
    print("it took --- %s seconds ---" % (time.time() - start_time)+'to go through everything for the ' + year_param + ' season')
    time.sleep(random.uniform(0,5)) #pause season pulls to reduce the chances of being throttled
    
    

In [51]:
df=pd.read_parquet(savedir+'df_2019-2020_data.gzip')

In [61]:
df_adv_box.drop(columns=['E_USG_PCT','E_PACE','E_OFF_RATING','E_DEF_RATING','E_NET_RATING'],inplace=True)

NameError: name 'df_adv_box' is not defined